In [2]:
import os
import cv2
#import pafy
import math
import random
import numpy as np
import datetime as dt
import tensorflow as tf
from collections import deque
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import plot_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Precision, Recall

In [3]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('MP_Data') 

# Actions that we try to detect
actions = np.array(['Hi', 'I am', 'From', 'Singapore', 'Paiseh', 'I', 'Cannot', 'Speak', 'But', 'Machine', 'Learning', 'Can', 'Help', 'Translate'])

# Thirty videos worth of data
no_sequences = 50

# Videos are going to be 30 frames in length
sequence_length = 30

# Folder start
start_folder = 0

In [4]:
label_map = {label:num for num, label in enumerate(actions)}

In [5]:
sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [6]:
X = np.array(sequences)

In [7]:
y = to_categorical(labels).astype(int)

In [11]:
loss = []
accuracies = []
precisions = []
recalls = []

kf = KFold(n_splits=5, shuffle=True)
for train_index, test_index in kf.split(X):
    
    np.random.shuffle(train_index)
    np.random.shuffle(test_index)
    
    features_train, features_test = X[train_index], X[test_index]
    labels_train, labels_test = y[train_index], y[test_index]
    
    # Construct the required LSTM model.
    model = Sequential() # changed activation to tanh
    model.add(LSTM(64, return_sequences=True, activation='tanh', recurrent_dropout=0.2, input_shape=(30,1662)))
    model.add(LSTM(128, return_sequences=True, activation='tanh', recurrent_dropout=0.2))
    model.add(LSTM(64, return_sequences=False, activation='tanh', recurrent_dropout=0.2))
    model.add(Dense(64, activation='tanh'))
    model.add(Dense(32, activation='tanh'))
    model.add(Dense(actions.shape[0], activation='softmax'))

    # Display the success message.
    print("Model Created Successfully!")
    
    # Create an Instance of Early Stopping Callback
    early_stopping_callback = EarlyStopping(monitor = 'val_loss', patience = 15, mode = 'min', restore_best_weights = True)

    model.compile(optimizer=Adam(learning_rate=1e-5), loss='categorical_crossentropy', metrics=['categorical_accuracy',Precision(),Recall()]) 
    
    model_training_history = model.fit(features_train, labels_train, epochs=300, batch_size = 4,
                                                     shuffle = True, validation_split = 0.2, 
                                                     callbacks = [early_stopping_callback])

    # Evaluate the trained model.
    model_evaluation_history = model.evaluate(features_test, labels_test)
    
    eval_loss, eval_acc, eval_prec, eval_rec = model_evaluation_history
    loss.append(eval_loss)
    accuracies.append(eval_acc)
    precisions.append(eval_prec)
    recalls.append(eval_rec)

Model Created Successfully!
Epoch 1/300
112/112 [==============================] - 35s 195ms/step - loss: 2.6332 - categorical_accuracy: 0.0781 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_loss: 2.6345 - val_categorical_accuracy: 0.0625 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/300
112/112 [==============================] - 18s 157ms/step - loss: 2.6112 - categorical_accuracy: 0.1384 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_loss: 2.6177 - val_categorical_accuracy: 0.0714 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 3/300
112/112 [==============================] - 19s 166ms/step - loss: 2.5819 - categorical_accuracy: 0.1429 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_loss: 2.5896 - val_categorical_accuracy: 0.0714 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 4/300
112/112 [==============================] - 18s 161ms/step - loss: 2.5389 - categorical_accuracy: 0.1518 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_l

112/112 [==============================] - 23s 206ms/step - loss: 1.8272 - categorical_accuracy: 0.4129 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_loss: 1.9303 - val_categorical_accuracy: 0.2946 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 32/300
112/112 [==============================] - 23s 202ms/step - loss: 1.7902 - categorical_accuracy: 0.4085 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_loss: 1.8898 - val_categorical_accuracy: 0.2946 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 33/300
112/112 [==============================] - 20s 182ms/step - loss: 1.7876 - categorical_accuracy: 0.4308 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_loss: 1.8744 - val_categorical_accuracy: 0.3393 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 34/300
112/112 [==============================] - 21s 192ms/step - loss: 1.7753 - categorical_accuracy: 0.4353 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_loss: 1.8561 - val_categorical_accurac

Epoch 63/300
112/112 [==============================] - 15s 133ms/step - loss: 1.3747 - categorical_accuracy: 0.6049 - precision: 0.9811 - recall: 0.1161 - val_loss: 1.5068 - val_categorical_accuracy: 0.4464 - val_precision: 1.0000 - val_recall: 0.0982
Epoch 64/300
112/112 [==============================] - 17s 156ms/step - loss: 1.3799 - categorical_accuracy: 0.5848 - precision: 0.9655 - recall: 0.1250 - val_loss: 1.4807 - val_categorical_accuracy: 0.4911 - val_precision: 1.0000 - val_recall: 0.0625
Epoch 65/300
112/112 [==============================] - 17s 153ms/step - loss: 1.3702 - categorical_accuracy: 0.6027 - precision: 0.9636 - recall: 0.1183 - val_loss: 1.4342 - val_categorical_accuracy: 0.5357 - val_precision: 1.0000 - val_recall: 0.0714
Epoch 66/300
112/112 [==============================] - 18s 165ms/step - loss: 1.3805 - categorical_accuracy: 0.6071 - precision: 0.9508 - recall: 0.1295 - val_loss: 1.4625 - val_categorical_accuracy: 0.5536 - val_precision: 1.0000 - val_rec

112/112 [==============================] - 21s 191ms/step - loss: 1.0473 - categorical_accuracy: 0.7232 - precision: 0.9661 - recall: 0.2545 - val_loss: 1.1693 - val_categorical_accuracy: 0.6518 - val_precision: 0.9259 - val_recall: 0.2232
Epoch 96/300
112/112 [==============================] - 20s 179ms/step - loss: 1.0434 - categorical_accuracy: 0.7545 - precision: 0.9661 - recall: 0.2545 - val_loss: 1.1630 - val_categorical_accuracy: 0.6071 - val_precision: 0.8519 - val_recall: 0.2054
Epoch 97/300
112/112 [==============================] - 20s 180ms/step - loss: 1.0334 - categorical_accuracy: 0.7366 - precision: 0.9750 - recall: 0.2612 - val_loss: 1.1131 - val_categorical_accuracy: 0.6429 - val_precision: 0.8846 - val_recall: 0.2054
Epoch 98/300
112/112 [==============================] - 18s 158ms/step - loss: 1.0374 - categorical_accuracy: 0.7277 - precision: 0.9520 - recall: 0.2656 - val_loss: 1.0967 - val_categorical_accuracy: 0.6696 - val_precision: 0.9259 - val_recall: 0.2232
E

112/112 [==============================] - 16s 144ms/step - loss: 0.8062 - categorical_accuracy: 0.8259 - precision: 0.9524 - recall: 0.4911 - val_loss: 0.9148 - val_categorical_accuracy: 0.7054 - val_precision: 0.8772 - val_recall: 0.4464
Epoch 128/300
112/112 [==============================] - 13s 120ms/step - loss: 0.7926 - categorical_accuracy: 0.8304 - precision: 0.9368 - recall: 0.5290 - val_loss: 0.9178 - val_categorical_accuracy: 0.7589 - val_precision: 0.8810 - val_recall: 0.3304
Epoch 129/300
112/112 [==============================] - 14s 125ms/step - loss: 0.8289 - categorical_accuracy: 0.8192 - precision: 0.9215 - recall: 0.4978 - val_loss: 1.0551 - val_categorical_accuracy: 0.6875 - val_precision: 0.7955 - val_recall: 0.3125
Epoch 130/300
112/112 [==============================] - 13s 119ms/step - loss: 0.8080 - categorical_accuracy: 0.8348 - precision: 0.9295 - recall: 0.5000 - val_loss: 0.8634 - val_categorical_accuracy: 0.7411 - val_precision: 0.9038 - val_recall: 0.419

...

112/112 [==============================] - 12s 104ms/step - loss: 0.5573 - categorical_accuracy: 0.9040 - precision_4: 0.9532 - recall_4: 0.7277 - val_loss: 0.6613 - val_categorical_accuracy: 0.8304 - val_precision_4: 0.9167 - val_recall_4: 0.6875
Epoch 146/300
112/112 [==============================] - 12s 105ms/step - loss: 0.5492 - categorical_accuracy: 0.9174 - precision_4: 0.9687 - recall_4: 0.7589 - val_loss: 0.6597 - val_categorical_accuracy: 0.8571 - val_precision_4: 0.9302 - val_recall_4: 0.7143
Epoch 147/300
112/112 [==============================] - 12s 105ms/step - loss: 0.5490 - categorical_accuracy: 0.8929 - precision_4: 0.9574 - recall_4: 0.7522 - val_loss: 0.5896 - val_categorical_accuracy: 0.8929 - val_precision_4: 0.9186 - val_recall_4: 0.7054
Epoch 148/300
112/112 [==============================] - 12s 109ms/step - loss: 0.5476 - categorical_accuracy: 0.9085 - precision_4: 0.9484 - recall_4: 0.7388 - val_loss: 0.6270 - val_categorical_accuracy: 0.8482 - val_precision

112/112 [==============================] - 17s 150ms/step - loss: 0.3908 - categorical_accuracy: 0.9219 - precision_4: 0.9660 - recall_4: 0.8884 - val_loss: 0.5141 - val_categorical_accuracy: 0.8750 - val_precision_4: 0.9038 - val_recall_4: 0.8393
Epoch 177/300
112/112 [==============================] - 16s 140ms/step - loss: 0.3870 - categorical_accuracy: 0.9375 - precision_4: 0.9662 - recall_4: 0.8929 - val_loss: 0.4759 - val_categorical_accuracy: 0.9018 - val_precision_4: 0.9300 - val_recall_4: 0.8304
Epoch 178/300
112/112 [==============================] - 16s 144ms/step - loss: 0.3965 - categorical_accuracy: 0.9353 - precision_4: 0.9541 - recall_4: 0.8817 - val_loss: 0.4748 - val_categorical_accuracy: 0.9018 - val_precision_4: 0.9216 - val_recall_4: 0.8393
Epoch 179/300
112/112 [==============================] - 14s 124ms/step - loss: 0.3973 - categorical_accuracy: 0.9397 - precision_4: 0.9553 - recall_4: 0.9062 - val_loss: 0.4677 - val_categorical_accuracy: 0.9196 - val_precision

112/112 [==============================] - 14s 121ms/step - loss: 0.2854 - categorical_accuracy: 0.9576 - precision_4: 0.9744 - recall_4: 0.9330 - val_loss: 0.3813 - val_categorical_accuracy: 0.8839 - val_precision_4: 0.9151 - val_recall_4: 0.8661
Epoch 208/300
112/112 [==============================] - 14s 124ms/step - loss: 0.2811 - categorical_accuracy: 0.9688 - precision_4: 0.9770 - recall_4: 0.9487 - val_loss: 0.3726 - val_categorical_accuracy: 0.9286 - val_precision_4: 0.9623 - val_recall_4: 0.9107
Epoch 209/300
112/112 [==============================] - 14s 123ms/step - loss: 0.3625 - categorical_accuracy: 0.9219 - precision_4: 0.9417 - recall_4: 0.9018 - val_loss: 0.6336 - val_categorical_accuracy: 0.8036 - val_precision_4: 0.8224 - val_recall_4: 0.7857
Epoch 210/300
112/112 [==============================] - 13s 118ms/step - loss: 0.2824 - categorical_accuracy: 0.9643 - precision_4: 0.9680 - recall_4: 0.9464 - val_loss: 0.3742 - val_categorical_accuracy: 0.9286 - val_precision

112/112 [==============================] - 14s 125ms/step - loss: 0.2152 - categorical_accuracy: 0.9732 - precision_4: 0.9818 - recall_4: 0.9621 - val_loss: 0.3124 - val_categorical_accuracy: 0.9286 - val_precision_4: 0.9444 - val_recall_4: 0.9107
Epoch 239/300
112/112 [==============================] - 14s 123ms/step - loss: 0.2116 - categorical_accuracy: 0.9688 - precision_4: 0.9750 - recall_4: 0.9576 - val_loss: 0.2636 - val_categorical_accuracy: 0.9464 - val_precision_4: 0.9464 - val_recall_4: 0.9464
Epoch 240/300
112/112 [==============================] - 15s 137ms/step - loss: 0.2321 - categorical_accuracy: 0.9598 - precision_4: 0.9683 - recall_4: 0.9531 - val_loss: 0.3456 - val_categorical_accuracy: 0.9286 - val_precision_4: 0.9450 - val_recall_4: 0.9196
Epoch 241/300
112/112 [==============================] - 16s 146ms/step - loss: 0.2067 - categorical_accuracy: 0.9732 - precision_4: 0.9819 - recall_4: 0.9688 - val_loss: 0.3269 - val_categorical_accuracy: 0.9196 - val_precision

112/112 [==============================] - 15s 136ms/step - loss: 0.1697 - categorical_accuracy: 0.9732 - precision_4: 0.9819 - recall_4: 0.9665 - val_loss: 0.3314 - val_categorical_accuracy: 0.9196 - val_precision_4: 0.9444 - val_recall_4: 0.9107
Epoch 270/300
112/112 [==============================] - 16s 145ms/step - loss: 0.1678 - categorical_accuracy: 0.9732 - precision_4: 0.9775 - recall_4: 0.9710 - val_loss: 0.3426 - val_categorical_accuracy: 0.9196 - val_precision_4: 0.9266 - val_recall_4: 0.9018
Epoch 271/300
112/112 [==============================] - 15s 134ms/step - loss: 0.1924 - categorical_accuracy: 0.9710 - precision_4: 0.9772 - recall_4: 0.9576 - val_loss: 0.3081 - val_categorical_accuracy: 0.9107 - val_precision_4: 0.9273 - val_recall_4: 0.9107
Epoch 272/300
112/112 [==============================] - 15s 134ms/step - loss: 0.1825 - categorical_accuracy: 0.9598 - precision_4: 0.9683 - recall_4: 0.9554 - val_loss: 0.3155 - val_categorical_accuracy: 0.9286 - val_precision

In [12]:
print("Loss:", loss)
print("Accuracies", accuracies)
print("Precisions", precisions)
print("Recalls", recalls)

Loss: [0.22031794488430023, 0.3187350630760193, 0.2534598708152771, 0.28669193387031555, 0.1639184206724167]
Accuracies [0.9857142567634583, 0.9642857313156128, 0.949999988079071, 0.9357143044471741, 0.9571428298950195]
Precisions [0.9928057789802551, 0.9772727489471436, 0.9701492786407471, 0.9333333373069763, 0.9640287756919861]
Recalls [0.9857142567634583, 0.9214285612106323, 0.9285714030265808, 0.8999999761581421, 0.9571428298950195]


In [13]:
print("Mean Loss:", sum(loss)/len(loss))
print("Mean Accuracies", sum(accuracies)/len(accuracies))
print("Mean Precisions", sum(precisions)/len(precisions))
print("Mean Recalls", sum(recalls)/len(recalls))

Mean Loss: 0.24862464666366577
Mean Accuracies 0.9585714221000672
Mean Precisions 0.9675179839134216
Mean Recalls 0.9385714054107666
